# Setup Cloud SQL - Python Connector
---

Install [cloud-sql-python connector](https://github.com/GoogleCloudPlatform/cloud-sql-python-connector#how-to-use-this-connector) using [pymysql](https://github.com/PyMySQL/PyMySQL) as the driver

In [ ]:
!pip install cloud-sql-python-connector[pymysql]

Authenticate using ***application default credentials***

In [ ]:
!gcloud auth application-default login

### Code

In [ ]:
# import connector
from google.cloud.sql.connector import connector, IPTypes

# import pandas
import pandas as pd

In [ ]:
conn = connector.connect(
    "dummy-329203:us-central1:dummy-mysql",
    "pymysql",
    ip_types=IPTypes.PUBLIC,
    user="root",
    password="CKnEx4mOtx2EADil",
    db="mysql"
)

In [ ]:
cursor = conn.cursor()
cursor.execute("SHOW COLUMNS FROM sys.sys_config")

columns = cursor.fetchall()
column_names = [col[0] for col in columns]

cursor.execute("SELECT * from sys.sys_config")
data = cursor.fetchall()

df = pd.DataFrame(data, columns=column_names)
df

,variable,value,set_time,set_by
0,diagnostics.allow_i_s_tables,OFF,2021-10-23 11:44:44,None
1,diagnostics.include_raw,OFF,2021-10-23 11:44:44,None
2,ps_thread_trx_info.max_length,65535,2021-10-23 11:44:44,None
3,statement_performance_analyzer.limit,100,2021-10-23 11:44:44,None
4,statement_performance_analyzer.view,None,2021-10-23 11:44:44,None
5,statement_truncate_len,64,2021-10-23 11:44:44,None


# Setup Cloud SQL - SQL Alchemy
---

Install [pymysql](https://docs.sqlalchemy.org/en/14/dialects/mysql.html#module-sqlalchemy.dialects.mysql.pymysql)

In [1]:
!pip install pymysql

     |████████████████████████████████| 43 kB 1.8 MB/s 


Authenticate using ***application default credentials***

In [2]:
!gcloud auth application-default login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=4Xe9tGk7IRrMI60rD3gwctBovYN5VN&prompt=consent&access_type=offline&code_challenge=sJDsR8K6KDwCqDgYa6dWJsBut06jIZoNwQ2Cbx7YGTw&code_challenge_method=S256

Enter verification code: 4/1AX4XfWidesrihGZMMQKwumuZntkOpWIc4NtOh81S_v19cJqTG0t5eH8fZ44

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to ADC. You might receive a "quota exceeded" or "API not enabled" error. Run $ gcloud auth application-defaul

Need to add new network if use public IP: `0.0.0.0/0`



### Code

In [3]:
# import create_engine from sql alchemy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

# import pandas
import pandas as pd

# import drive
from google.colab import drive

Mount gdrive folder to get SSL files

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


Build connection without SSL

In [ ]:
path_to_ssl = '/content/drive/MyDrive/Colab Notebooks/SSL'

engine = create_engine(
  URL.create(
    drivername="mysql+pymysql",
    username="root",
    password="CKnEx4mOtx2EADil",
    host="34.72.191.193",
    port=3306,
    database="mysql"
  )
)

conn = engine.connect()

Build connection with SSL

In [6]:
path_to_ssl = '/content/drive/MyDrive/Colab Notebooks/creds/SSL'

ssl_files={
    "ssl_ca": f"{path_to_ssl}/server-ca.pem",
    "ssl_cert": f"{path_to_ssl}/client-cert.pem",
    "ssl_key": f"{path_to_ssl}/client-key.pem"
}

engine = create_engine(
  URL.create(
    drivername="mysql+pymysql",
    username="root",
    password="CKnEx4mOtx2EADil",
    host="34.72.191.193",
    port=3306,
    database="mysql"
  ),
  connect_args=ssl_files
)

conn = engine.connect()

In [7]:
query_get_cols = "SHOW COLUMNS FROM sys.sys_config"
query_result = conn.execute(query_get_cols)
column_names = query_result.fetchall()
columns = [col[0] for col in column_names]

query_get_data = "select * from sys.sys_config"
query_result = conn.execute(query_get_data)
data = query_result.fetchall()

df = pd.DataFrame(data, columns=columns)
df

,variable,value,set_time,set_by
0,diagnostics.allow_i_s_tables,OFF,2021-10-23 11:44:44,None
1,diagnostics.include_raw,OFF,2021-10-23 11:44:44,None
2,ps_thread_trx_info.max_length,65535,2021-10-23 11:44:44,None
3,statement_performance_analyzer.limit,100,2021-10-23 11:44:44,None
4,statement_performance_analyzer.view,None,2021-10-23 11:44:44,None
5,statement_truncate_len,64,2021-10-23 11:44:44,None


# Setup Git Repo
---

Change directory to the working directory in gdrive

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/github/web-scraping-to-cloud-sql

/content/drive/MyDrive/Colab Notebooks/github/web-scraping-to-cloud-sql


Print current working directory

In [ ]:
!pwd
!ls

/content/drive/MyDrive/Colab Notebooks/github/web-scraping-to-cloud-sql
mysql-cloud-sql.ipynb


Init .git directory

In [ ]:
!git init

Reinitialized existing Git repository in /content/drive/MyDrive/Colab Notebooks/github/web-scraping-to-cloud-sql/.git/


Retrieve **github access token** from .txt file in gdrive 

In [ ]:
drive.mount('/content/drive')
file = open('/content/drive/MyDrive/Colab Notebooks/creds/github_token.txt')
github_token = file.read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Add remote repo

In [ ]:
username = 'firdausraginda'
repo_name = 'beautiful-soup-to-cloud-sql'
!git remote add origin https://{github_token}@github.com/{username}/{repo_name}.git

Remove remote just for debugging purpose

In [ ]:
!git remote remove origin
!git remote

Set config user email & user name

In [ ]:
!git config user.email "ragindafirdaus01@gmail.com"
!git config user.name "firdausraginda"

In [ ]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   mysql-cloud-sql.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


Add modified files to staging

In [ ]:
!git add .

Commit to .git directory

In [ ]:
!git commit -m "initial commit"

[master 1117540] initial commit
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite mysql-cloud-sql.ipynb (95%)


Check list of remotes & branches

In [ ]:
!git remote
!git branch

origin
* master


Push to master

In [ ]:
!git push -u origin master

Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 794 bytes | 397.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/firdausraginda/beautiful-soup-to-cloud-sql.git
   832aa03..1117540  master -> master
Branch 'master' set up to track remote branch 'master' from 'origin'.


# Web Scraping Using Beautiful Soup
---